<a href="https://colab.research.google.com/github/ayaartay/A-B-test-wine-quality-analysis/blob/master/Compustat.v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install linearmodels

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import pylab

import pandas as pd
import statsmodels.formula.api as sm
import statsmodels.stats.sandwich_covariance as sw
import numpy as np
import statsmodels as statsmodels
from linearmodels import PanelOLS, FamaMacBeth

from linearmodels import PanelOLS
from linearmodels.panel import generate_panel_data

In [120]:
# Importing the data
df = pd.read_csv (r'/content/compustat.csv')

In [121]:

df = df[df['fyear'] != 1994.0]
df.head(2)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,prstkc,pstk,pstkl,sale,txditc,xint,costat,prcc_f,loc,sic
1,1004,19960531,1995.0,INDL,C,D,STD,AIR,AAR CORP,USD,...,1.552,0.0,0.0,504.990,30.68,10.616,A,22.125,USA,5080.0
2,1004,19970531,1996.0,INDL,C,D,STD,AIR,AAR CORP,USD,...,8.080,0.0,0.0,589.328,32.56,10.786,A,31.000,USA,5080.0


In [122]:
df.shape

(266912, 33)

In [123]:
df_us = df[df['loc'] == 'USA']
df_us.shape

(208749, 33)

In [124]:
df_year = df_us.groupby(['fyear'])['gvkey'].count()
df_year.head()

fyear
1995.0    11599
1996.0    11633
1997.0    11333
1998.0    11319
1999.0    11364
Name: gvkey, dtype: int64

In [ ]:
df_year.plot(kind='line',x='fyear',y='gvkey')

In [126]:
# Creating 2 lagged variables for at and prcc_f, respectively

df_us['lat'] = df_us.shift(1)['at']
df_us['lprice'] = df_us.shift(1)['prcc_f']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# Creating variables for the given financial ratios

df_us['blev1'] = (df_us['dlc'] + df_us['dltt']) / df_us['at'] #book leverage 1
df_us['blev2'] = df_us['lt'] / df_us['at']                    #book leverage 2
df_us['mve'] = df_us['csho'] * df_us['prcc_f']                #mv of equity
df_us['mlev'] = (df_us['dlc'] + df_us['dltt']) / (df_us['dlc'] + df_us['dltt'] + df_us['pstk'] + df_us['mve']) #market leverage
df_us['m2b'] = (df_us['dlc'] + df_us['dltt']) / (df_us['dlc'] + df_us['dltt'] + df_us['pstk'] + df_us['mve']) #market to book 
df_us['agrow'] = df_us['at']/df_us['lat']-1 #asset growth
df_us['atang'] = df_us['ppent']/df_us['at'] #asset tangibility
df_us['roe'] = df_us['ni']/df_us['ceq'] #return on equity
df_us['pm'] = df_us['ni']/df_us['sale'] #profit margin
df_us['capx2at'] = df_us['capx']/df_us['at'] #capex ratio
df_us['dvyld'] = (df_us['dv']/df_us['csho'])/(df_us['lprice']/df_us['csho']) #dividend yield
df_us['dvpay'] = df_us['dv']/df_us['ni'] #dividend payout ratio
df_us['tpay'] = (df_us['dv']+df_us['prstkc'])/df_us['ni'] #total payout ratio
df_us['ebit'] = df_us['ebit']/df_us['xint'] #EBIT interest coverage
df_us['cash'] = df_us['che']/df_us['at'] #cash holdings
df_us['profit'] = (df_us['oibdp']/df_us['at']) #profitability

In [ ]:
# Creating a subsample cleaned from NaNs and infs

ratio_list = ['blev1','blev2','mve','mlev','m2b','agrow','atang','roe','pm','capx2at','dvyld','dvpay','tpay','ebit','cash','profit']
for ratio_name in ratio_list:
  df_us[str(ratio_name)+str('_c')] = df_us[ratio_name].replace([np.inf, -np.inf], np.nan)

ratio_list_c = ['blev1_c','blev2_c','mve_c','mlev_c','m2b_c','agrow_c','atang_c','roe_c','pm_c','capx2at_c','dvyld_c','dvpay_c','tpay_c','ebit_c','cash_c','profit_c']
df_c = df_us.dropna()
print(df_c.shape)
df_c.head()

In [129]:
groups = df_c.fyear.unique()
groups.sort()
type(groups)

numpy.ndarray

In [ ]:
# Winsorizing the financial ratios

from scipy.stats.mstats import winsorize

# groupping 'fyear' by years

groups = df_c.fyear.unique()
groups.sort()

# using the cleaned columns stored as ratio_list_c above
for year in groups:
  for i in df_c[ratio_list_c]:
    df_c[str(i)+str('w')] = winsorize(df_c[i], limits=[0.01, 0.01], inplace=True) 

In [131]:
# creating a list for winsorized variables
ratios_w = ['blev1_cw','blev2_cw','mve_cw','mlev_cw','m2b_cw','agrow_cw','atang_cw','roe_cw','pm_cw','capx2at_cw','dvyld_cw','dvpay_cw','tpay_cw','ebit_cw','cash_cw','profit_cw']

In [132]:
# Summary statistics for the financial ratios

df_c[ratios_w].describe()

,blev1_cw,blev2_cw,mve_cw,mlev_cw,m2b_cw,agrow_cw,atang_cw,roe_cw,pm_cw,capx2at_cw,dvyld_cw,dvpay_cw,tpay_cw,ebit_cw,cash_cw,profit_cw
count,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000,75232.000000
mean,0.370451,0.791920,2149.035359,0.275943,0.275943,0.552142,0.277838,0.008302,-1.375164,0.055864,1.168528,0.123858,0.293260,8.634049,0.147029,-0.072113
std,0.614475,1.285772,6924.243225,0.256680,0.256680,3.244237,0.250226,1.493782,7.594408,0.068913,4.401233,0.419556,0.962038,160.242282,0.186295,0.711775
min,0.000000,0.077723,0.356650,0.000000,0.000000,-0.984277,0.000000,-8.072905,-64.666667,0.000000,0.000000,-1.139428,-3.146945,-834.966667,0.000000,-5.255814
25%,0.085776,0.381354,22.970047,0.057400,0.057400,-0.081118,0.073885,-0.070454,-0.097120,0.013964,0.000000,0.000000,0.000000,-1.446055,0.020504,0.005666
50%,0.247808,0.566057,153.729770,0.204910,0.204910,0.039000,0.196718,0.083673,0.021003,0.033611,0.000000,-0.000000,0.000000,2.482318,0.071244,0.093866
75%,0.428093,0.754558,976.460895,0.431941,0.431941,0.195276,0.425153,0.178159,0.071092,0.069061,0.202208,0.077030,0.378975,9.092082,0.197957,0.153316
max,4.897691,10.855602,51632.337190,0.959859,0.959859,28.295154,0.912071,7.443268,0.675822,0.393911,33.453410,2.668524,5.782328,974.196721,0.861386,0.405113


Question 1.6. Commentary


a)



b)



**Part 2. EDA**


Q4

In [133]:
#4a. Estimating the model y = a + b*x + e where y is firm's book leverage (1) and x is firm's profits  

import statsmodels.formula.api as sm

y = df_c['blev1_cw']
x = df_c['profit_cw']

ols = sm.ols('y ~ x', data=df_c).fit(use_t=True)
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                 3.918e+04
Date:                Tue, 13 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:59:38   Log-Likelihood:                -54343.
No. Observations:               75232   AIC:                         1.087e+05
Df Residuals:                   75230   BIC:                         1.087e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3340      0.002    182.927      0.000       0.330       0.338
x             -0.5052      0.003   -197.928      0.000      -0.510      -0.500
==============================================================================
Omnibus:                    51613.879   Durbin-Watson:                   0.862
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2017514.654
Skew:                           2.808   Prob(JB):                         0.00
Kurtosis:                      27.740   Cond. No.                         1.42
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [134]:
# Making Standard Errors heteroscedasticity robust (HC1)

robust_ols = sm.ols(formula='y ~ x', data=df_c).fit(cov_type='HC1', use_t=True)
robust_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                     3092.
Date:                Tue, 13 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:59:43   Log-Likelihood:                -54343.
No. Observations:               75232   AIC:                         1.087e+05
Df Residuals:                   75230   BIC:                         1.087e+05
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3340      0.002    222.474      0.000       0.331       0.337
x             -0.5052      0.009    -55.610      0.000      -0.523      -0.487
==============================================================================
Omnibus:                    51613.879   Durbin-Watson:                   0.862
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2017514.654
Skew:                           2.808   Prob(JB):                         0.00
Kurtosis:                      27.740   Cond. No.                         1.42
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [162]:
#4b. Estimating the model by adding firm and year fixed effects using statsmodels

# linearmodels needs the index to be entity/date.
df_ci = df_c.set_index(['gvkey', 'fyear'])

FE = PanelOLS(df_ci.blev1_cw, df_ci['profit_cw'],
              entity_effects = True,
              time_effects=True
              ).fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
FE.summary

Dep. Variable:,blev1_cw,R-squared:,0.2028
Estimator:,PanelOLS,R-squared (Between):,0.3958
No. Observations:,75232,R-squared (Within):,0.2045
Date:,"Tue, Sep 13 2022",R-squared (Overall):,0.2911
Time:,19:17:10,Log-likelihood,-2.335e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.628e+04
Entities:,11194,P-value,0.0000
Avg Obs:,6.7207,Distribution:,"F(1,64017)"
Min Obs:,1.0000,,
Max Obs:,21.000,F-statistic (robust):,513.87


In [161]:
#4c. OLS firm-by-firm

# Filtering the firms with at least 10 non-missing observations, i.e data is present for at least half of the 1995-2015 time range

min_freq = 10
model_vars = ['blev1_cw', 'profit_cw']

def filter_by_freq(df: pd.DataFrame, column: str, min_freq: int) -> pd.DataFrame:
    freq = df[column].value_counts()
    frequent_values = freq[freq >= min_freq].index
    return df[df[column].isin(frequent_values)]

for var in model_vars:
  filter_by_freq(df_c, var, min_freq)

In [ ]:

ols = sm.ols(formula='y ~ x', data=df_c).fit(cov_type='HC1', use_t=True)
robust_ols.summary()

In [165]:
# Running the OLS firb-by-firm (Warning: this command took XXX minutes!)

def OLSfirm (df, y, x):
    y = df_c['blev1_cw']
    x = df_c['profit_cw']
    #x['intercept'] = 1.
    result = sm.ols(formula='y ~ x', data=df_c).fit(cov_type='HC1', use_t=True)
    return result.params

df_c.groupby('gvkey').apply(OLSfirm, 'blev1_cw', ['profit_cw'])

,Intercept,x
gvkey,,
1004,0.334021,-0.505178
1013,0.334021,-0.505178
1019,0.334021,-0.505178
1021,0.334021,-0.505178
1034,0.334021,-0.505178
...,...,...
287462,0.334021,-0.505178
287616,0.334021,-0.505178
289735,0.334021,-0.505178


In [ ]:
#4d. Plottingthe histogram for beta estimates

beta_hats = result.params[1]
plt.hist(beta_hats)
plt.show() 

In [ ]:
# Summary statistics for beta estimates

beta_hats.describe()

In [ ]:
# Summary statistics for the 4 quartile subsamples 



4e. Commentary

Average/ Median estimates close to the full-sample ones?

Is the variation large?

How does the estimates depend on  the MVE?

**Part 3 Bankrupcies in Compustat**